# パーザコンビネータ

## ラムダ式

In [1]:
 def succ(x):
    return x+1

print(succ(0)) # 1 と表示される

1


In [2]:
#  succ の型を調べる
type(succ)

function

In [3]:
# succ を f に代入
f = succ
print(f(0))

1


In [4]:
# ラムダ式による関数定義
succ = lambda x : x + 1
print(succ(0))

1


## 高階関数

In [5]:
def map(f, xs) :
    mapped = []
    for x in xs:
        mapped.append(f(x))
    return mapped

In [6]:
 # 関数名による map 
 map(succ, [1,2,3])

[2, 3, 4]

In [7]:
# ラムダ式による map
map(lambda x: x * 2, [1,2,3])

[2, 4, 6]

## クロージャ

In [8]:
def counter():
    x=0 #ローカル変数
    def f() :
        nonlocal x
        x=x+1
        return x
    return f # 定義された関数を返す

In [9]:
# クロージャの実行例
c = counter()
print(c())
print(c())
print(c())

1
2
3


## パーサコンビネーター

In [10]:
def pAny():
    def match(x):
        if len(x) > 0:
            y = x[1:]
            return y
        return None
    return match

In [11]:
dot = pAny() # パーザ関数を dot とする
dot('ab')

'b'

In [13]:
print(dot(''))

None


In [14]:
def pChar(text):
    def match(x):
        if x.startswith(text):
            return x[len(text):]
        return None
    return match

In [15]:
#解析表現'a'で入力"ab"をマッチ 
a = pChar('a')
a('ab')

'b'

In [18]:
#解析表現'b'で入力"ab"をマッチ 
b = pChar('b')
print(b('ab'))

None


In [19]:
def pRange(cs):
    def match(x):
        if len(x) >= 1 and x[0] in cs:
            y = x[1:]
            return y
        return None
    return match

In [20]:
abc = pRange('abc')
abc('ab')

'b'

In [21]:
def pNot(e):
    # e が文字列のとき， pChar(e) に変換
    e = pChar(e) if isinstance(e, str) else e
    def match(x):
        if not e(x):
            return x
        return None
    return match
 

In [22]:
 #!'a'で入力"ab"をマッチ 
 not_a = pNot(pChar('a')) 
 print(not_a('ab'))

None


In [25]:
def pMany(e):
    # e が文字列のとき， pChar(e) に変換
    e = pChar(e) if isinstance(e, str) else e
    def match(x):
        y = e(x)
        while y != None:
            x = y
            y = e(x)
        return x
    return match

In [26]:
# 'a'*で入力"aaaabb"をマッチ
many_a = pMany(pChar('a')) 
many_a('aaaabb')

'bb'

In [27]:
def pSeq(e1, e2):
    e1 = pChar(e1) if isinstance(e1, str) else e1
    e2 = pChar(e2) if isinstance(e2, str) else e2
    def match(x):
        y = e1(x)
        if y == None:
            return None
        return e2(y)
    return match

In [28]:
#  'a' 'b'で入力"ab"をマッチ
ab = pSeq('a', 'b') 
ab('ab')

''

In [29]:
def pSeq(*es):
    # 文字列'a'はpChar('a')に変換する
    es = [pChar(e) if isinstance(e, str) else e for e in es]

    if len(es) == 1: # 要素が一つならそのまま 
        return es[0]
    
    def match(x):
        for e in es:
            x = e(x)
            if x is None:
                return None
        return x
    return match

In [30]:
# 'a' 'a'* 'b'
pSeq('a', pMany('a'), 'b')


<function __main__.pSeq.<locals>.match(x)>

In [31]:
def pOre(e1, e2):
    e1 = pChar(e1) if isinstance(e1, str) else e1
    e2 = pChar(e2) if isinstance(e2, str) else e2
    def match(x):
        y = e1(x)
        if y != None:
            return y
        return e2(x)
    return match

In [32]:
# 'a' / 'c'で入力"ab"をマッチ
a_or_b = pOre('a', 'b') 
a_or_b('ab')

'b'

In [33]:
def pRef(peg, name):
    if name in peg: # すでにパーザ関数が定義済みなら
        return peg[name] # そのまま定義済み関数を返す 
    def match(x):
        e = peg[name]
        return e(x)
    return match

In [34]:
peg = {}
peg['A'] = pOre(pSeq('a', pRef(peg, 'A')), '')
peg['A']('aaaabb')

'bb'

## 10.3 パーザコンビネータとPEG文法

In [35]:
peg = {}
peg["A"] = pOre(pSeq('a', pRef(peg, 'A'), 'a'), 'a') 
print(peg['A']('a'))
print(peg['A']('aa'))
print(peg['A']('aaa')) 
print(peg['A']('aaaa')) 
print(peg['A']('aaaaa'))


a

aaa
aa


In [38]:
peg = {}
peg['EXPR'] = pSeq(pRef(peg, 'PROD'), pMany(pSeq('+', pRef(peg, 'PROD')))) 
peg['PROD'] = pSeq(pRef(peg, 'TERM'), pMany(pSeq('*', pRef(peg, 'TERM'))))
peg['TERM'] = pSeq(pRef(peg, 'DIGIT'), pMany(pRef(peg, 'DIGIT'))) 
peg['DIGIT'] = pRange('0123456789')

peg['EXPR']('1+2*3')

''

## 12.2 メモ化


In [39]:
def pRef(peg, A):
    def match(x):
        o = peg[A](x)
        return o
    return match

In [ ]:
# メモあり

M = {}
def pRef(peg, A):
    def match(x):
        if (A, x) in M:
            return M[(A, x)]
        o = peg[A](x)
        M[(A, x)] = o
        return o
    return match


In [40]:
peg = {}
peg['A'] = pOre(pSeq('a', pRef(peg, 'A')), '')
peg['A']('aaaabb')

'bb'